# Construction d'un système de recommandation

Nous avons décidé d'orienter notre projet sur la recommendation de films.
En effet durant ce confinement, nous avons eu le temps de visionner beaucoup de films,
mais nous nous sommes rendus compte que nous passions quasiment autant de temps
à choisir le film qu'à le regarder. D'où la nécessité de créer un système de re-
commendations afin d'optimiser notre temps de visionnage.
Nous avons chercher une base de données assez exploitable afin de mener à bien
notre projet. Nous nous sommes basés sur la base de données de 'The Movies Dataset'.

# Différents systèmes de recommandation

- [x] memory-based (user- et item- based)
- [ ] hybride
- [ ] model-based (matrix factorisation, optimisation avec descente de gradient)
- [x] popularity based = moyenne simple
- [ ] user-centered linear approach = descente de gradient (même pb d'opti que model-based)
- [ ] clustering (??)

In [205]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import re
from ast import literal_eval

## Fetching and cleaning data

Nous utilisons deux tables de données. L'une, *movies_metadata.csv*, contient une liste de films et des informations relativesau genre, date de sortie etc. 

### Informations sur les films

In [206]:
movies = pd.read_csv("movies_metadata.csv")
movies.head()

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [207]:
movies.describe()

,revenue,runtime,vote_average,vote_count
count,4.546000e+04,45203.000000,45460.000000,45460.000000
mean,1.120935e+07,94.128199,5.618207,109.897338
std,6.433225e+07,38.407810,1.924216,491.310374
min,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,85.000000,5.000000,3.000000
50%,0.000000e+00,95.000000,6.000000,10.000000
75%,0.000000e+00,107.000000,6.800000,34.000000
max,2.787965e+09,1256.000000,10.000000,14075.000000


In [208]:
def filter_correct_id(word):
    if re.fullmatch(r'[0-9]+', word):
        return word
    return "wrong_id"

In [209]:
# don't re-run
movies = movies[~movies.id.duplicated()]
movies.id = movies.id.apply(filter_correct_id)
movies = movies[movies.id != "wrong_id"]
movies.id = movies.id.astype('int64')

### Avis des utilisateurs

In [210]:
ratings = pd.read_csv("ratings_small.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [211]:
# ne pas re-run !
ratings = ratings.drop(columns=['timestamp'])
ratings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [212]:
ratings[(ratings['userId'] == 1) & (ratings['movieId'] == 31)]

,userId,movieId,rating
0,1,31,2.5


In [213]:
print(min(ratings.rating), max(ratings.rating))
ratings.describe()
ratings.dtypes

0.5 5.0


userId       int64
movieId      int64
rating     float64
dtype: object

In [214]:
nbPers = len(ratings.userId.unique())
nbMovi = len(ratings.movieId.unique())

## User Based Recommandation

Pour ce système, nous n'avons que besoin des notations des utilisateurs et des titres des films associés. Nous allons translater les notes afin que la moyenne des notes pour chaque utilisateur se trouve à 0. Par abus de langage nous appelons ces nouvelles notes les notes *normalisées*. 

In [215]:
print(len(ratings.movieId.unique()))
ratings_small = ratings.loc[ratings['userId'] <= 50]

9066


In [216]:
def mean_user(df, uid):
    '''
    Retourne la moyenne des notes données par l'utilisateur d'id uid
    '''
    n = df.loc[ratings['userId'] == uid].count().loc['userId']
    s = df.loc[ratings['userId'] == uid].sum().loc['rating']
    return s / n

In [217]:
def normalize_user(df):
    '''
    Ajoute une colonne dans la dataframe df contenant les notes normalisées des utilisateurs
    '''
    mean = df.loc[:, ['userId']].drop_duplicates()
    mean['mu'] = mean['userId'].map(lambda uid : mean_user(df, uid))
    mean = mean.set_index('userId')
    df['rating_norm'] = df[['userId', 'rating']].apply(lambda row : row['rating'] -  mean.loc[int(row['userId'])]['mu'], axis=1)

In [218]:
normalize_user(ratings_small)
print(ratings_small.head())

   userId  movieId  rating  rating_norm
0       1       31     2.5        -0.05
1       1     1029     3.0         0.45
2       1     1061     3.0         0.45
3       1     1129     2.0        -0.55
4       1     1172     4.0         1.45


/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Regrouper les utilisateurs dans des peer-group

Pour déterminer si deux utilisateurs se ressemblent en termes de goûts, nous utilisons un taux de corrélation sur les avis données. Nous allons comparer quatres taux de corrélations différents. Le premier ```cor()``` calcule le taux de corrélation classique donné par la formule :
$$
cor(u, v) = \frac{\sum_{k \in I_{uv}} s_{uk} s_{vk}}{\sqrt{\sum_{k \in I_{uv}} s_{uk}^2}\sqrt{\sum_{k \in I_{uv}} s_{vk}^2}}
$$

Le taux de corrélation ajusté ```cor_adj()``` permet de ne pas donner trop d'importance aux films populaires que beaucoup de personnes ont vu.

Le taux de correlation calculé par ```cor_dis()``` permet de ne pas donner une correlation trop élevée si les deux utilisateurs n'ont pas donné assez d'avis sur des films en commun. 

Enfin la fonction ```cor_dis_adj()``` fait un mélange des deux dernières amélioration : il filtre les films trop populaire et n'apporte de l'importance seulement si deux personnes ont données leur avis sur un certain nombre de films.

In [219]:
def cor(u, v, df, Iuv):
    su = df.loc[(df['userId'] == u) & (df['movieId'].isin(Iuv['movieId']))].rating_norm
    sv = df.loc[(df['userId'] == v) & (df['movieId'].isin(Iuv['movieId']))].rating_norm
    su = np.array(su)
    sv = np.array(sv)
    
    return np.dot(su, sv) / math.sqrt(np.dot(su, su) * np.dot(sv, sv))

In [220]:
def cor_adj(u, v, df, Iuv):
    nb_rat = df.loc[:, ['movieId', 'rating']].groupby(['movieId']).count()
    
    sum_up = 0
    sum_down_u = 0
    sum_down_v = 0
    for movie in Iuv.movieId.unique() :
        suk = df.loc[(df['userId'] == u) & (df['movieId'] == movie), ['rating_norm']]
        svk = df.loc[(df['userId'] == v) & (df['movieId'] == movie), ['rating_norm']]
        suk, svk = float(suk), float(svk)
        
        sum_up += suk * svk / nb_rat.at[movie, 'rating']
        sum_down_u += suk**2 /  nb_rat.at[movie, 'rating']
        sum_down_v += svk**2 /  nb_rat.at[movie, 'rating']
    return sum_up / math.sqrt(sum_down_u * sum_down_v)

In [221]:
def cor_dis(u, v, df, Iuv):
    beta = 20
    correlation = cor(u, v, df, Iuv)
    return correlation * min(len(Iuv), beta)/beta

In [222]:
def cor_dis_adj(u, v, df, Iuv):
    beta = 20
    correlation = cor_adj(u, v, df, Iuv)
    return correlation * min(len(Iuv), beta)/beta

**Test des taux de corrélation sur les utilisateurs 2 et 3 qui ont 8 films en communs**

In [223]:
u, v = 2, 3
df = ratings_small
Iu = df.loc[df['userId'] == u, ['movieId']]
Iv = df.loc[df['userId'] == v, ['movieId']]
Iuv = Iu.join(Iv.set_index('movieId'), on='movieId', how='inner')
print(Iuv)

print(cor(u, v, df, Iuv))
# print(cor_adj(u, v, df))
print(cor_dis(u, v, df, Iuv))
# print(cor_dis_adj(u, v, df))

    movieId
27      110
49      296
57      356
64      377
79      527
88      588
91      592
92      593
-0.016060945830838957
-0.006424378332335582


Nous construisons maintenant la matrice de correlation. Puisqu'une telle matrice est symétrique, nous avons préféré utiliser une dataframe à deux entrées et ne stocker la corrélation pour un couple qu'une seule fois.

In [224]:
def uu_matrix(df, cor_fct=cor):
    '''
    Retourne la dataframe des taux de corrélations des utilisateurs de df
    '''
    correlation = []
    users = df.userId.unique()
    couples = []
    for i in range(len(users)):
        u = users[i]
        if not u % 20 : 
            print('user:', u, end='')
        for j in range(i + 1, len(users)):
            v = users[j]
            Iu = df.loc[df['userId'] == u, ['movieId']]
            Iv = df.loc[df['userId'] == v, ['movieId']]
            Iuv = Iu.join(Iv.set_index('movieId'), on='movieId', how='inner')
            if Iuv.size :
                couples.append((u, v))
                correlation.append(cor_fct(u, v, df, Iuv))
    index = pd.MultiIndex.from_tuples(couples, names=['u', 'v'])
    cor = pd.DataFrame(correlation, index=index, columns=['correlation'])
    return cor

In [225]:
def get_cor(u, v, cm):
    '''
    Retourne le taux de correlation entre u et v stocké dans cm
    '''
    if u > v :
        u, v = v, u
    index = list(cm.index.values)
    if (u, v) in index :
        return float(cm.loc[(u, v)])
    return float('nan')

In [226]:
cm_small = uu_matrix(ratings_small)

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


user: 20user: 40

In [227]:
cm_small.head()

correlation
u v              
1 4      0.042137
  5     -1.000000
  7     -0.752427
  9      1.000000
  15     0.043773

In [228]:
print(get_cor(1, 4, cm_small))
print(get_cor(4, 1, cm_small))
print(get_cor(2, 1, cm_small))

0.042136808375910856
0.042136808375910856
nan


Nous allons maintenant prédire la note qu'un utilisateur **u** donnerait à un film *m . Pour cela, nous allons faire la somme des notes données à l'item *i* par les k utilisateurs plus proches de u qui ont donné une note à m. Cette somme sera pondérée par les coéfficients de corrélations.

$$
\hat{\sigma}_{um} = \mu_u + \frac{\sum_{v \in P_u(m)} s_{vm} \cdot cor(u, v)}{\sum_{v \in P_u(m)} |cor(u,v)|}
$$

Le peer-group de l'utilisateur u pour le film m est l'ensemble des k utilisateurs qui ont donné une note au film m les plus proche de l'utilisateur u en terme de taux de corrélation. 

In [229]:
def peers(user, movie, k, cm, df):
    '''
    Retourne les k utilisateurs du peer-group de (user, movie)
    '''
    top = [(float('-inf'), user)] * k
    df_movie = df.loc[df['movieId'] == movie, ['userId', 'rating_norm']]
    for v in df_movie.userId.unique():
        taux = get_cor(user, v, cm)
        if taux > top[-1][0] :
            top += [(taux, v)]
            top.sort(reverse=True)
            top = top[:-1]
    return [t[1] for t in top]

In [230]:
def predict(user, movie, k, cm, df):
    mu = mean_user(user)
    peer_group = peers(user, movie, k, cm, df)
    sum_up, sum_down = 0, 0
    for v in peer_group:
        cor = get_cor(user, v, cm)
        if not math.isnan(cor):
            svm = df.loc[(df['userId'] == v) & (df['movieId'] == movie), 'rating_norm']
            svm = 0 if len(svm) <= 0 else float(svm)
            sum_up += svm * cor
            sum_down += abs(cor)
    sum_down = 1 if sum_down == 0 else sum_down
    return mu + sum_up / sum_down

In [231]:
user = 85
movie = 10
k = 4

friends = peers(user, movie, k, cm_small, ratings_small)
p = predict(user, movie, k, cm_small, ratings_small)
print(friends)
print(p)
print(df.loc[(df['userId'].isin(friends)) & (df['movieId'] == 10)])

TypeError: mean_user() missing 1 required positional argument: 'uid'

## Item Based Recommandation

Nous allons maintenant construire un système de recommandation basé sur les items. Dans le système user-based, pour prédire la note donnée par l'utilisateur *user* au film *movie*, on regardait les notes donnés à ce même film *movie* par des utilisateurs similaires à *user*. De manière analogue, dans le système item-based, pour prédire la note donnée par user à *movie*, on regarde les notes données par *user* à des films similaires à *movie*.

In [ ]:
ratings_mm = ratings.loc[ratings['movieId'] <= 100]

**Normalisation des notes données aux films par leur moyenne**

In [ ]:
def mean_movie(df, mid):
    '''
    Retourne la moyenne des notes données au film d'id mid
    '''
    n = df.loc[ratings['movieId'] == mid].count().loc['movieId']
    s = df.loc[ratings['movieId'] == mid].sum().loc['rating']
    return s / n

In [ ]:
def normalize_movie(df):
    '''
    Ajoute une colonne dans la dataframe df contenant les notes normalisées des films
    '''
    mean = df.loc[:, ['movieId']].drop_duplicates()
    mean['mu'] = mean['movieId'].map(lambda mid : mean_movie(df, mid))
    mean = mean.set_index('movieId')
    df['norm_movie'] = df[['movieId', 'rating']].apply(lambda row : row['rating'] -  mean.loc[int(row['movieId'])]['mu'], axis=1)

In [ ]:
normalize_movie(ratings_mm)

In [ ]:
ratings_mm = ratings_mm.sort_values(by='movieId')
ratings_mm.head()

In [ ]:
def cor_movie(u, v, df, Iuv):
    su = df.loc[(df['movieId'] == u) & (df['userId'].isin(Iuv['userId']))].norm_movie
    sv = df.loc[(df['movieId'] == v) & (df['userId'].isin(Iuv['userId']))].norm_movie
    su = np.array(su)
    sv = np.array(sv)
    
    return np.dot(su, sv) / math.sqrt(np.dot(su, su) * np.dot(sv, sv))

In [ ]:
def mm_matrix(df, cor_fct=cor_movie):
    '''
    Retourne la dataframe des taux de corrélations des utilisateurs de df
    '''
    correlation = []
    movies = df.movieId.unique()
    couples = []
    for i in range(len(movies)):
        u = movies[i]
        if not i % 20 : 
            print('movie number:', i, end=' ')
        for j in range(i + 1, len(movies)):
            v = movies[j]
            Iu = df.loc[df['movieId'] == u, ['userId']]
            Iv = df.loc[df['movieId'] == v, ['userId']]
            Iuv = Iu.join(Iv.set_index('userId'), on='userId', how='inner')
            if Iuv.size :
                couples.append((u, v))
                correlation.append(cor_fct(u, v, df, Iuv))
    index = pd.MultiIndex.from_tuples(couples, names=['u', 'v'])
    cor = pd.DataFrame(correlation, index=index, columns=['correlation'])
    return cor

In [ ]:
cm_small = mm_matrix(ratings_mm)

In [ ]:
cm_small.head()

In [ ]:
def peers_movie(user, movie, k, cm, df):
    '''
    Retourne les k films du peer-group de (movie, user)
    '''
    top = [(float('-inf'), movie)] * k
    df_user = df.loc[df['userId'] == user, ['movieId', 'norm_movie']]
    for v in df_user.movieId.unique():
        taux = get_cor(movie, v, cm)
        if taux > top[-1][0] :
            top += [(taux, v)]
            top.sort(reverse=True)
            top = top[:-1]
    return [t[1] for t in top]

In [ ]:
def predict(user, movie, k, cm, df):
    mu = mean_movie(movie)
    sum_up, sum_down = 0, 0
    peer_group = peers_movie(user, movie, k, cm, df)
    for v in peer_group:
        cor = get_cor(movie, v, cm)
        if not math.isnan(cor):
            svm = df.loc[(df['movieId'] == v) & (df['userId'] == user), 'norm_movie']
            svm = 0 if len(svm) <= 0 else float(svm)
            sum_up += svm * cor
            sum_down += abs(cor)
    sum_down = 1 if sum_down == 0 else sum_down
    return mu + sum_up / sum_down

In [ ]:
cm = cm_small
df = ratings_mm
user = 85
movie = 10
k = 4

friends = peers_movie(user, movie, k, cm, df)
p = predict(user, movie, k, cm, df)
print(friends)
print(p)
print(df.loc[(df['movieId'].isin(friends)) & (df['userId'] == user)])

## Factorisation du user- et item- based

Dans un système user-based, le ptype (pour peer-type) est userId et le otype (pour other-type) est movieId. Dans un système item-based c'est l'inverse. 

In [ ]:
ratings_small = ratings.loc[(ratings['userId'] <= 100) & (ratings['movieId'] <= 100)]
ratings_small.head()

### Normalisation

In [ ]:
def mean_base(df, pid, base):
    '''
    Retourne la moyenne des notes données par l'utilisateur d'id uid
    '''
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
    n = df.loc[ratings[ptype] == pid].count().loc[ptype]
    s = df.loc[ratings[ptype] == pid].sum().loc['rating']
    return s / n

In [ ]:
pid = 4
print(mean_base(ratings_small, pid, 'movie'))
print(mean_base(ratings_small, pid, 'user'))

In [ ]:
def normalize(df, base='user'):
    '''
    Ajoute une colonne dans la dataframe df contenant les notes normalisées des utilisateurs
    '''
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
    mean = df.loc[:, [ptype]].drop_duplicates()
    mean['mu'] = mean[ptype].map(lambda pid : mean_base(df, pid, base))
    mean = mean.set_index(ptype)
    df['rating_norm'] = df[[ptype, 'rating']].apply(lambda row : row['rating'] -  mean.loc[int(row[ptype])]['mu'], axis=1)
    df.sort_values(by=ptype, inplace=True)

In [ ]:
normalize(ratings_small, 'movie')

In [ ]:
ratings_small.head()

### Matrice de correlation

# Big question : on donne quoi comme correlation si su et/ou sv est nul ? J'ai mis 0 par défaut mais bon ...

# Big question 2 : un gars ([ici](https://www.ethanrosenthal.com/2015/11/02/intro-to-collaborative-filtering/)) fait cette fonction pour calculer la matrice plus rapidement 

In [ ]:
def fast_similarity(ratings, kind='user', epsilon=1e-9):
    # epsilon -> small number for handling dived-by-zero errors
    if kind == 'user':
        sim = ratings.dot(ratings.T) + epsilon
    elif kind == 'item':
        sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

In [ ]:
def cor(df, u, v, Iuv, base):
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
    su = df.loc[(df[ptype] == u) & (df[otype].isin(Iuv[otype]))].rating_norm
    sv = df.loc[(df[ptype] == v) & (df[otype].isin(Iuv[otype]))].rating_norm
    su = np.array(su)
    sv = np.array(sv)
    up = np.dot(su, sv)
    down = math.sqrt(np.dot(su, su) * np.dot(sv, sv))
    if up == 0 or down == 0:
        return 0
    return up/ down

In [ ]:
def cor_matrix(df, cor_fct=cor, base='user'):
    '''
    Retourne la dataframe des taux de corrélations des ptype de df
    '''
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
    correlation = []
    peers = df.userId.unique() if base == 'user' else df.movieId.unique()
    couples = []
    for i in range(len(peers)):
        u = peers[i]
        if not u % 20 : 
            print('peer :', u, end='\n')
        for j in range(i + 1, len(peers)):
            v = peers[j]
            Iu = df.loc[df[ptype] == u, [otype]]
            Iv = df.loc[df[ptype] == v, [otype]]
            Iuv = Iu.join(Iv.set_index(otype), on=otype, how='inner')
            if Iuv.size :
                couples.append((u, v))
                correlation.append(cor_fct(df, u, v, Iuv, base))
    index = pd.MultiIndex.from_tuples(couples, names=['u', 'v'])
    cor = pd.DataFrame(correlation, index=index, columns=['correlation'])
    return cor

In [ ]:
cm_small = cor_matrix(ratings_small, base='movie')

In [ ]:
cm_small.head()

### Prédire une note pour un couple p/o donné

In [ ]:
def peers(df, cm, k, p, o, base='user'):
    '''
    Retourne les k films du peer-group de (movie, user)
    '''
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
    top = [(float('-inf'), p)] * k
    df_peers = df.loc[df[ptype] == p, [otype, 'rating_norm']]
    peers = df_peers.iloc[0].unique()
    for v in peers:
        taux = get_cor(p, v, cm)
        if taux > top[-1][0] :
            top += [(taux, v)]
            top.sort(reverse=True)
            top = top[:-1]
    return [t[1] for t in top]

In [ ]:
def predict(df, cm, k, p, o, base='user'):
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
    mu = mean_base(df, p, base)
    peer_group = peers(df, cm, k, p, o, base)
    sum_up, sum_down = 0, 0
    for v in peer_group:
        cor = get_cor(p, v, cm)
        if not math.isnan(cor):
            svm = df.loc[(df[ptype] == v) & (df[otype] == o), 'rating_norm']
            svm = 0 if len(svm) <= 0 else float(svm)
            sum_up += svm * cor
            sum_down += abs(cor)
    sum_down = 1 if sum_down == 0 else sum_down
    return mu + sum_up / sum_down

In [ ]:
df = ratings_small
cm = cm_small
user = 85
movie = 10
k = 4

friends = peers(df, cm, k, user, movie, base='user')
p = predict(df, cm , k, user, movie, base='user')
print(friends)
print(p)
print(df.loc[(df['movieId'].isin(friends)) & (df['userId'] == user)])

In [ ]:
full_rat = ratings
normalize(full_rat)
full_rat.head()

In [ ]:
full_cm = uu_matrix(full_rat)

### Recommand top 10 movies to user

## Top films par genres

### Selection des genres

In [ ]:
movies = movies.rename(columns={'id' : 'movieId'})
movies = movies.loc[:, ['movieId', 'genres', 'title']]
movies.head()

In [ ]:
def simplify_genre(l):
    if len(l) <= 0 :
        return []
    if isinstance(l[0], dict):
        return [d['name'] for d in l]
    return l

movies['genres'] = movies['genres'].apply(lambda x: literal_eval(x) if isinstance(x, str) else x).apply(simplify_genre)
movies.head()

### Normaliser les notes sur toutes les données

In [ ]:
movies_small = movies.loc[movies['movieId'] <= 100]
movies_small['mean_rating'] = movies_small['movieId'].apply(lambda x: mean_base(ratings, x, 'movie'))

In [ ]:
movies_small.head()

In [ ]:
movies_small = movies_small.sort_values(by='mean_rating', ascending=False)

In [ ]:
movies_small.head()

In [ ]:
# 1 genre par ligne
serie = movies_small.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
serie.name = 'genres'
df_simpl = movies_small.drop('genres', axis=1).join(serie)
df_simpl.head()

In [ ]:
def top(df_simpl, genres):
    # selection des films par genre
    df_genres = df_simpl.loc[df_simpl['genres'].isin(genres)]
    df_genres = df_genres.sort_values(by='mean_rating', ascending=False)
    return df_genres

In [ ]:
top(df_simpl, ['Drama', 'Comedy']).head()

## Recommandation hybride

## Model-based recommendation system

Aide :
https://towardsdatascience.com/introduction-to-recommender-systems-6c66cf15ada

La matrice des notes user-item $R$ est partiellement vide. Ainsi réduire les dimensions de la matrice pourrait améliorer la complexité de nos algorithmes. Une méthode que nous pourrions avoir envie d'utiliser est la décomposision en valeurs singulières : $R = U_{svd} \Sigma V_{svd}$. Cependant cette méthode ne s'applique pas ici étant donné que $R$ n'est pas complète et qu'on a besoin de réaliser des calculs algébriques avec $R$ pour trouver la décomposition.

On considère donc un modèle dans lequel il existe des attributs décrivants les films et les préférences des utilisateurs. La matrice $R$ peut alors être factorisée en produit de deux matrices $U$ et $V$ représentant respectivement les utilisateurs et les items :

$$
R \approx U \times V^T
$$

avec $R \in \mathbb{R}^{n \times m}$ la matrice des notes user-item, $U \in \mathbb{R}^{n \times \ell}$ la matrice des users, $V \in \mathbb{R}^{m \times \ell}$ la matrice des items et $\ell$ le nombre d'attributs. Pour faire un rapprochement avec la SVD, on peut considerer que $U = U_{svd} \Sigma^{1/2}$ et $V = \Sigma^{1/2} V_{svd}$. On note $U_i$ les lignes de $U$ et $V_j$ les lignes de $V$ :
$
U = \left[ \begin{array}{c} U_1 \\ \vdots \\ U_n \end{array} \right]
$ et 
$
V = \left[ \begin{array}{c} V_1 \\ \vdots \\ V_m \end{array} \right]
$
avec $U_i^T, V_j^T \in \mathbb{R^\ell}$.

Dans ce modèle, chaque note $R_{ij}$ associée à un couple user-item $(i, j)$ est le résultat du produit scalaire entre la ligne associée au user $i$ dans $U$ et la ligne associée au item $j$ dans $V$ : $R_{ij} = U_i \cdot V_j^T$. Une fois les matrices $U$ et $V$ apprises, pour prédire une note il suffira de faire le produit scalaire entre les lignes associées.

Trouver $U$ et $I$ revient à minimiser l'erreur entre la note prédite $U_i \cdot V_j^T$ et la véritable note $R_{ij}$. Il s'agit du problème de minimisation suivant, avec $E = \{(i, j) \mbox{ | } R_{ij} \mbox{ connue}\}$ :

$$
(U, V) = argmin_{(U, V)} \sum_{(i, j) \in E} [U_i \cdot V_j^T - R_{ij}]^2
$$

qui est équivalent à:

$$
(U, V) = argmin_{(U, V)} \frac{1}{2}\sum_{(i, j) \in E} [U_i \cdot V_j^T - R_{ij}]^2 + \lambda (\|U_i\|^2 + \|V_j\|^2)
$$

Le terme de droite est un terme régulateur, de paramètre $\lambda$ à ajuster, permettant de prévenir un overfitting.

Pour résoudre ce problème, nous allons utiliser une méthode de descente de gradient.


*Pour résoudre ce problème, on peut utiliser une méthode de descente de gradient. Nous allons ensuite optimiser cette méthode en utilisant d'abord des batch, puis en se réduisant à un problème de moindre carré en fixant alternativement les matrices $U$ et $V$.*

### Descente de gradient (à pas constant)

Dans notre [cours d'optimisation](https://www.ceremade.dauphine.fr/~gontier/enseignement.html) donné par David Gontier, nous avons étudié différentes méthodes de descente de gradient de complexité et d'optimalité différentes. Cependant il nous semble qu'utiliser une version simple à pas $\tau$ constant suffit. Il sera possible de régler cet hyper-paramètre par validation croisée. 

Notre fonction objective est la suivante :
$$
F(U, V) := \sum_{(i, j) \in E} \frac{1}{2}[U_i \cdot V_j^T - R_{ij}]^2 + \frac{\lambda}{2} (\|U_i\|^2 + \|V_j\|^2)
$$

Dans une descente de gradient classique, à chaque itération on met à jour $U$ et $V$ suivant la formule 
$
(U, V) = (U, V) - \tau \nabla F(U, V)
$. Cependant, dans notre cas nous n'allons pas mettre à jour toutes les lignes de $U$ et $V$ simultanément. En effet, puisque la somme dans $F$ ne se fait que sur les couples $(i, j)$ pour lesquels la note est connue, nous allons seulement mettre à jour le couple $(U_i, V_j)$ associé en itérant sur tous les couples $(i, j) \in E$. 

Pour une note $R_{ij}$, on a 
$
\frac{\partial F}{\partial U_i} = V_j^T (U_i \cdot V_j^T - R_{ij}) + \lambda U_i
$
 et 
$
\frac{\partial F}{\partial V_j} = Ui (U_i \cdot V_j^T - R_{ij}) + \lambda V_j
$
donc on peut mettre à jour les lignes $U_i$ et $V_j$ selon les formules 
$$
U_i = Ui - \tau [V_j^T (U_i \cdot V_j^T - R_{ij}) + \lambda U_i]\\
V_j = V_j - \tau [Ui (U_i \cdot V_j^T - R_{ij}) + \lambda V_j]
$$

In [232]:
R = ratings.loc[(ratings['userId'] <= 50) & (ratings['movieId'] <= 100)]

In [233]:
R.head()

,userId,movieId,rating
0,1,31,2.5
20,2,10,4.0
21,2,17,5.0
22,2,39,5.0
23,2,47,4.0


In [234]:
n = len(R.userId.unique())
m = len(R.movieId.unique())
print('n:', n, 'm:', m)

n: 44 m: 65


In [235]:
a = R.movieId.unique()
a.sort()
print(a)
R.userId.unique()

[  1   2   3   4   5   6   7   8   9  10  11  14  16  17  18  19  20  21
  22  23  24  25  26  29  31  32  34  35  36  39  40  41  42  44  45  47
  48  50  52  55  57  58  60  62  63  64  68  69  70  73  74  76  78  79
  81  82  85  86  88  89  92  94  95  97 100]


array([ 1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 13, 15, 16, 17, 18, 19, 20,
       21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 43, 44, 46, 47, 48, 49, 50])

Nous remarquons que tous les entiers entre 1 et $n$ (ou $m$) ne sont pas nécessairement utlisés par les id des users (ou des movies). Ceci peut être du au nettoyage effecté par exemple. Puisque nous aimerions utiliser des numpy array dans nos calculs, il va être nécessaire d'avoir la correspondant entre les id et les indices utilisés dans les numpy array (que nous allons appeler rang). Pour cela, utilisons simplement une liste contenant les id et dont l'indice dans la liste d'un id donné correspondra au rang. Pour trouver l'id à partir d'un rang il suffira de faire un simple extraction, pour trouver le rang à partir d'un id on utilisera la méthode `index()`.

In [236]:
user_rank = R.userId.unique().tolist()
movie_rank = R.movieId.unique().tolist()

La matrice $R$ étant vide, nous n'allons pas utiliser de matrice pour la représenter et garderons la dataframe qui ne contient que les notes connues. Nous allons également avoir besoin d'écrire une fonction `get_rat()` qui permet d'accéder à la note d'un couple de rang dans la dataframe des notes. Nous utilisons également une fonction `known()` pour construire l'ensemble $E$.

In [237]:
def get_rat(R, i, j):
    '''
    Retourne la note de rang (i, j) dans la dataframe R
    '''
    uid, mid = user_rank[i], movie_rank[j]
    return float(R.loc[(R['userId'] == uid) & (R['movieId'] == mid), 'rating'])

In [238]:
def known(R):
    '''
    Retourne l'ensemble des indices (i, j) pour lesquels la note est connue dans R
    '''
    ids = set(R.loc[:, ['userId', 'movieId']].itertuples(index=False, name=None))
    E = set(map(lambda t : (user_rank.index(t[0]), movie_rank.index(t[1])), ids ))
    return E

Nous pouvons à présent écrire la fonction résolvant notre problème de minimisation. Remarquons qu'elle modifie les valeurs de $U$ et $V$ en place.

In [254]:
def calculate_rmse(U, V, E):
    predicted = np.dot(U, V.T) # the predicted rating matrix

    rmse = 0
    nb_instances = 0
    for (i, j) in E :
        rmse += (predicted[i, j] - get_rat(R, i, j)) ** 2 
        nb_instances += 1
    return np.sqrt(rmse / nb_instances)

In [284]:
def descenteGradient(U, V, tau, tol=1e-3, Niter=100):
    E = known(R)
    last_rmse = 0
    for n in range(Niter):
        
        if not n % 10 :
            print(n, end=' ')
            
        rmse = calculate_rmse(U, V, E)
        if abs(rmse - last_rmse) < tol:
            return U, V
        last_rmse = rmse

        for (i, j) in E :
            gradU = V[j].T * (np.dot(U[i], V[j].T) - get_rat(R, i, j)) + lamb * U[i]
            gradV = U[i] * (np.dot(U[i], V[j].T) - get_rat(R, i, j)) + lamb * V[j]
            U[i] = U[i] - tau * gradU
            V[j] = V[j] - tau * gradV
    print("Erreur, l’algorithme n’a pas convergé après", Niter ," itérations")
    return U, V

In [285]:
# hyper-paramètres à tuned
ell = 2
tau = 1/10
lamb = 0
U, V = np.random.rand(n, ell), np.random.rand(m, ell)
print(U[:5, :5])
print(V[:5, :5])

[[0.34413137 0.62593147]
 [0.61774419 0.69362482]
 [0.45472714 0.22173626]
 [0.11030463 0.65624302]
 [0.62039583 0.11146004]]
[[0.23735894 0.47901343]
 [0.83425896 0.24624606]
 [0.98152465 0.11705095]
 [0.36846962 0.50369504]
 [0.14071548 0.05347817]]


In [286]:
descenteGradient(U, V, tau)
rmse = calculate_rmse(U, V, E)

0 10 20 30 

In [287]:
print(rmse)

0.5821155633808284


## Linear model : content-based

On remarque que si $U$ ou $V$ est fixé, la fonction objective devient quadratique. Or nous connaissons des algorithmes efficaces pour minimiser des fonctions quadratiques. De plus, une matrice d'attributs des films peuvent être données puisque 